In [1]:
# code that divides covid confirmed and covid deaths by population 
import pandas as pd 
import numpy as np

In [2]:
jhuconfirmed_us = pd.read_csv('../processed_data/csse_data/confirmed_US_zip.csv')
jhudeaths_us= pd.read_csv('../processed_data/csse_data/deaths_US_zip.csv')

city_population_df = pd.read_csv('../raw_data/population_data/PEP_2018_PEPANNRES_with_ann.csv' , encoding='latin-1')

In [3]:
city_population_df.drop(city_population_df.index[0],inplace = True)

In [4]:
#city_state = city_population_df['GEO.display-label'].str.split(',')
#city_state = city_state.iloc[1:]
#city_population_df['city'] = city_state
row_list = [] 
for index, rows in city_population_df.iterrows():
    city = city_population_df['GEO.display-label'][index].split(',')[0].title()
    row_list.append(city)

In [5]:
city_population_df['city'] = row_list

In [6]:
def fix_nans(array): 
    for i,element in enumerate(array):
        array[i] = np.nan_to_num(array[i])
    return array 

In [7]:
population = city_population_df['respop72018']
cities = city_population_df['city']
pop_dict = dict(zip(cities, population)) 
confirmed = fix_nans(jhuconfirmed_us['Admin2'].map(pop_dict).array)
deaths = fix_nans(jhudeaths_us['Admin2'].map(pop_dict).array)

In [8]:
# now create one without city and without town
row_list = [] 

for index, rows in city_population_df.iterrows():
    city = city_population_df['GEO.display-label'][index].split(',')[0][:-5].title()
    row_list.append(city)
city_population_df['city'] = row_list

population = city_population_df['respop72018']
cities = city_population_df['city']
pop_dict = dict(zip(cities, population)) 



confirmed_2 = fix_nans(jhuconfirmed_us['Admin2'].map(pop_dict).array)
deaths_2 = fix_nans(jhudeaths_us['Admin2'].map(pop_dict).array)

In [9]:
# now create one without Village
row_list = [] 

for index, rows in city_population_df.iterrows():
    city = city_population_df['GEO.display-label'][index].split(',')[0][:-8].title()
    row_list.append(city)
city_population_df['city'] = row_list

population = city_population_df['respop72018']
cities = city_population_df['city']
pop_dict = dict(zip(cities, population)) 



confirmed_3 = fix_nans(jhuconfirmed_us['Admin2'].map(pop_dict).array)
deaths_3 = fix_nans(jhudeaths_us['Admin2'].map(pop_dict).array)

In [10]:
# add up the city/Town and the ones without city/town (there should be no overlap)
total_confirmed_col = confirmed.astype(int) + confirmed_2.astype(int) + confirmed_3.astype(int)
total_deaths_col = deaths.astype(int) + deaths_2.astype(int) + confirmed_3.astype(int)

In [11]:
jhuconfirmed_us.insert(2, 'population_2018', total_confirmed_col)
jhudeaths_us.insert(2, 'population_2018', total_deaths_col)

In [12]:
jhuconfirmed_us[1000:1200]

,UID,iso2,population_2018,county,iso3,code3,FIPS,Admin2,Province_State,Country_Region,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1000,84021005,US,234,Denali Borough,USA,840,21005.0,Anderson,Kentucky,US,...,3,3,3,3,3,4,5,5,7,6
1001,84021007,US,1046,Monroe County,USA,840,21007.0,Ballard,Kentucky,US,...,1,1,1,1,2,2,4,3,3,4
1002,84021009,US,0,NaN,USA,840,21009.0,Barren,Kentucky,US,...,1,1,5,5,5,5,6,6,6,6
1003,84021011,US,2916,Mason County,USA,840,21011.0,Bath,Kentucky,US,...,1,1,2,2,3,3,3,3,3,3
1004,84021013,US,488,Los Angeles County,USA,840,21013.0,Bell,Kentucky,US,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,84023027,US,533,Columbia County,USA,840,23027.0,Waldo,Maine,US,...,19,27,29,29,29,30,39,42,42,42
1196,84023029,US,6577,Nevada County,USA,840,23029.0,Washington,Maine,US,...,1,1,1,1,1,2,2,2,2,2
1197,84023031,US,8235,York County,USA,840,23031.0,York,Maine,US,...,137,143,144,151,152,156,164,170,175,177
1198,84024001,US,1717,Coos County,USA,840,24001.0,Allegany,Maryland,US,...,10,11,13,15,17,17,20,26,33,33


In [13]:
jhuconfirmed_us.to_csv('../processed_data/csse_data/confirmed_US_county_population.csv', index =None)
jhudeaths_us.to_csv('../processed_data/csse_data/deaths_US_county_population.csv', index =None)